# Managing Data Twitter Hadoop Application
### File Date: 2019-08-18

Import libraries

In [3]:
import tweepy
import numpy as np
import pandas as pd
import datetime
import operator
from datetime import date
from textblob import TextBlob
from datetime import datetime, timedelta

Define keys (keys have been ommitted for privacy reasons)

In [4]:
consumer_key= 'ommitted for security'
consumer_secret= 'ommitted for security'
access_token='ommitted for security'
access_token_secret='ommitted for security'

Get authorization

In [5]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

Define variables

In [7]:
max_tweets = 1000
today = date.today().strftime("%Y-%m-%d")
yesterday = date.today() - timedelta(1)
start_date = yesterday.strftime("%Y-%m-%d")

Retrieve tweets

In [8]:
celeb_tweets = [status for status in tweepy.Cursor(api.search, q = ['@katyperry OR @justinbieber OR @BarackObama OR @rihanna OR @taylorswift13 OR @ladygaga OR @TheEllenShow OR @cristiano OR @timberlake OR @ArianaGrande'], since = start_date, until = today, lang="en").items(max_tweets)]

Retrieve only user ant text

In [9]:
user_tweet = [['From :', tweet.user.screen_name, ', Text :', tweet.text] for tweet in celeb_tweets]

Convert to data frame

In [20]:
# Convert tweets to df
tweets_df = pd.DataFrame(data = user_tweet, columns = ['user', 'name', 'tweet', 'text'])
# Take a look at the data
tweets_df.head()

,user,name,tweet,text
0,From :,unholykidrauhl,", Text :",RT @lordsdrew: his voice gives me goosebumps @...
1,From :,alwsjustincj,", Text :",RT @billboardcharts: .@justinbieber on this we...
2,From :,RitaTaeTae,", Text :",RT @billboardcharts: .@BTS_twt is officially N...
3,From :,commonguy123,", Text :","RT @Peoples_Pundit: Iowa has not ""traditionall..."
4,From :,getmeddl,", Text :",nove @justinbieber @DanAndShay\n #10000Hours ...


Get the mention counts for each celebrity

In [13]:
mention_counts = [tweets_df.text.str.contains('@katyperry').sum(),
tweets_df.text.str.contains('@justinbieber').sum(),
tweets_df.text.str.contains('@BarackObama').sum(),
tweets_df.text.str.contains('@rihanna').sum(),
tweets_df.text.str.contains('@taylorswift13').sum(),
tweets_df.text.str.contains('@ladygaga').sum(),
tweets_df.text.str.contains('@TheEllenShow').sum(),
tweets_df.text.str.contains('@cristiano').sum(),
tweets_df.text.str.contains('@timberlake').sum(),
tweets_df.text.str.contains('@ArianaGrande').sum()]

Create mention count data frame

In [16]:
# Define column values
celeb_counts_df = {'celebrity': ['Katy Perry', 'Justin Bieber', 'Barack Obama', 'Rihanna', 'Taylor Swift', 'Lady Gaga', 'Ellen DeGeneres', 'Cristiano Ronaldo',
'Justin Timberlake', 'Ariana Grande'],
'number_of_tweets': mention_counts}
# Convert to data frame
df = pd.DataFrame(celeb_counts_df, columns = ['celebrity', 'number_of_tweets'])
# Take a look at the data
df.head()

,celebrity,number_of_tweets
0,Katy Perry,1
1,Justin Bieber,375
2,Barack Obama,252
3,Rihanna,23
4,Taylor Swift,76


Who is being tweeted the most about?

In [17]:
most_tweeted_celeb = df.loc[df['number_of_tweets'] == df['number_of_tweets'].max(), 'celebrity'].iloc[0]
print(most_tweeted_celeb, "has been tweeted the most.")

Justin Bieber has been tweeted the most.


Who has tweeted the most about this celebrity?

In [18]:
# Get tweets for the celebrity
public_tweets = [status for status in tweepy.Cursor(api.search, q = most_tweeted_celeb, since = start_date, until = today, lang = 'en').items(max_tweets)]
# Keep the user
user_list = [['From :', tweet.user.screen_name] for tweet in public_tweets]
# Convert to data frame
tweets = pd.DataFrame(data = user_list, columns = ['user', 'name'])
# Identify the most frequent user
most_frequent_tweeter = tweets['name'].value_counts().idxmax()
print ("@", most_frequent_tweeter, "has tweeted", most_tweeted_celeb, "the most.")

@ PawelPolandSwag has tweeted Justin Bieber the most.


Return summary

In [19]:
print ("The most tweeted celebrity in the past day is", most_tweeted_celeb, "and @", most_frequent_tweeter, "has tweeted", most_tweeted_celeb, "the most. Below is a table with the number of tweets for our celebrities of interest.")
print(df)

The most tweeted celebrity in the past day is Justin Bieber and @ PawelPolandSwag has tweeted Justin Bieber the most. Below is a table with the number of tweets for our celebrities of interest.
           celebrity  number_of_tweets
0         Katy Perry                 1
1      Justin Bieber               375
2       Barack Obama               252
3            Rihanna                23
4       Taylor Swift                76
5          Lady Gaga                45
6    Ellen DeGeneres                30
7  Cristiano Ronaldo                 3
8  Justin Timberlake                 0
9      Ariana Grande                66
